Paso a paso del noteboook de ejemplo:

In [24]:
import numpy as np
import pandas as pd
from itertools import cycle

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

from matplotlib import pyplot as plt

Importamos el df de housing.csv  y eliminamos las filas con algún na así cómo la columna 'ocean_proximity'. Que es la única que no es numérica.

In [31]:
df = pd.read_csv('housing.csv')
df = df.drop(columns=['ocean_proximity']).dropna()
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0
...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0


Eliminamos las casas con valor mayor a 500000

In [32]:
df = df.loc[(df['median_house_value'] < 500000), :]

df


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0
...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0


Creamos la columna 'y' con el cuantil (20%) de precio al cual pertenece la casa.

In [33]:
df['y'] = pd.qcut(df['median_house_value'], 5, labels=[0, 1, 2, 3, 4])

df

/tmp/ipykernel_32479/2809974913.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['y'] = pd.qcut(df['median_house_value'], 5, labels=[0, 1, 2, 3, 4])


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,y
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,4
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,4
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,4
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,4
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,4
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,0
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,0


Esconde la columna 'median_house_value'

In [34]:
df = df.drop(columns='median_house_value')

df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,y
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,4
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,4
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,4
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,4
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,4
...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,0
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,0


Divide el df en train y en test

In [35]:
train, test = train_test_split(df, test_size=0.3, random_state=42)

train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,y
9909,-122.30,38.31,34.0,1797.0,395.0,1162.0,407.0,3.4550,1
18709,-122.40,40.58,43.0,1455.0,300.0,747.0,279.0,2.7857,0
9298,-122.56,38.03,34.0,1887.0,290.0,815.0,283.0,6.5249,4
174,-122.24,37.80,52.0,996.0,228.0,731.0,228.0,2.2697,1
16676,-120.68,35.14,34.0,3100.0,617.0,1155.0,542.0,3.0938,3
...,...,...,...,...,...,...,...,...,...
11990,-117.53,33.97,34.0,1293.0,215.0,774.0,217.0,3.8906,1
12679,-121.38,38.55,26.0,1532.0,264.0,781.0,285.0,4.6944,1
5691,-118.24,34.23,43.0,1061.0,208.0,514.0,208.0,6.0100,3
876,-122.01,37.56,6.0,3028.0,778.0,1531.0,736.0,4.4259,2


In [36]:
test

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,y
9538,-120.71,37.39,11.0,1479.0,341.0,1476.0,327.0,3.2721,0
20231,-119.27,34.26,23.0,3578.0,753.0,1455.0,649.0,4.1898,4
9526,-123.16,39.13,33.0,1320.0,303.0,1048.0,303.0,1.7813,0
4222,-118.26,34.12,52.0,2290.0,520.0,1278.0,485.0,3.8393,3
20008,-119.06,36.15,20.0,1282.0,273.0,852.0,247.0,1.6354,0
...,...,...,...,...,...,...,...,...,...
6609,-118.15,34.19,38.0,1750.0,411.0,1398.0,409.0,2.3967,2
20591,-121.58,39.15,34.0,1376.0,376.0,702.0,317.0,1.4946,0
6156,-117.95,34.09,18.0,1179.0,324.0,1296.0,331.0,2.8510,1
14897,-117.07,32.59,21.0,1779.0,466.0,1327.0,488.0,1.6007,0


Crea y entrena el scaler. Para eso crea un Objeto de la clase StandardScaler (para estandarizar los datos) y lo entrena con el df train. Esto guarda los estadísticos necesarios para estandarizar los datos más adelante.

In [37]:
scaler = StandardScaler()
scaler.fit(train.drop(columns='y'))

StandardScaler()

In [42]:
train_estandarizados = pd.DataFrame(scaler.transform(train.drop(columns='y')))
train_estandarizados

,0,1,2,3,4,5,6,7
0,-1.359631,1.228200,0.450477,-0.375939,-0.341006,-0.240975,-0.243047,-0.131793
1,-1.409469,2.280953,1.171147,-0.534481,-0.567710,-0.602504,-0.579332,-0.562096
2,-1.489208,1.098346,0.450477,-0.334217,-0.591574,-0.543265,-0.568823,1.841893
3,-1.329729,0.991679,1.891817,-0.747262,-0.739528,-0.616442,-0.713321,-0.893841
4,-0.552268,-0.241943,0.450477,0.228098,0.188765,-0.247073,0.111628,-0.364014
...,...,...,...,...,...,...,...,...
13608,1.017605,-0.784551,0.450477,-0.609580,-0.770551,-0.578983,-0.742220,0.148261
13609,-0.901129,1.339505,-0.190119,-0.498786,-0.653619,-0.572885,-0.563569,0.665036
13610,0.663761,-0.663972,1.171147,-0.717130,-0.787255,-0.805483,-0.765865,1.510856
13611,-1.215103,0.880375,-1.791608,0.194720,0.572969,0.080481,0.621310,0.492414


Instanciamos el objeto PCA y lo entrenamos con los datos de train estandarizados. Posteriormente aplicamos PCA al df de train. Entonces en vez de 8 variables, tendremos 5.

In [44]:
pca_entrenado = PCA(n_components=5)
pca_entrenado.fit(train_estandarizados)
new_space = pca_entrenado.transform(train_estandarizados)

pd.DataFrame(new_space)

,0,1,2,3,4
0,-0.875873,1.756316,-0.168376,0.209482,0.132394
1,-1.668570,2.494629,-0.747676,0.412365,-0.092252
2,-1.133939,1.668596,1.603565,1.191363,0.107149
3,-2.017467,1.511847,-1.373908,0.918390,0.050776
4,-0.008388,0.259736,-0.503733,0.281075,-0.340138
...,...,...,...,...,...
13608,-1.269237,-1.412357,0.097283,0.242200,0.012530
13609,-1.173629,1.435089,0.920479,-0.040795,0.064749
13610,-1.541726,-1.124798,0.926305,1.525439,-0.076564
13611,1.005215,1.533025,1.195647,-1.128807,-0.098025


En este, entrenem el modelo del DecisionTreeClassifier en les dades estandaritzades i la reducció de dimensionalitat aplicada  per a que predica la columna 'y' del df train

In [46]:
cl_model = DecisionTreeClassifier(criterion='entropy', min_samples_split=50, random_state=42)
cl_model.fit(new_space, train.loc[:, 'y'])

DecisionTreeClassifier(criterion='entropy', min_samples_split=50,
                       random_state=42)

Ya está el modelo entrenado. Solo queda aplicar a los datos el modelo y que el DecisionTree los clasifique.